<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Bus_Route_Optimization0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 55.9 MB/s eta 0:00:00


In [2]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('bus routes')

# indices for companies and routes
company = [*range(0,6)]
route = [*range(0,8)]

# bids
c = [[0,	8200,	7800,	5400,	0,	3900,	0,	0],
    [7800,	8200,	0,	6300,	0,	3300,	4900,	0],
    [0,	4800,	0,	0,	0,	4400,	5600,	3600],
    [0,	0,	8000,	5000,	6800,	0,	6700,	4200],
    [7200,	6400,	0,	3900,	6400,	2800,	0,	3000],
    [7000,	5800,	7500,	4500,	5600,	0,	6000,	4200]]

# Valid set of tuples
A = []
for i in company:
    for j in route:
        if c[i][j] > 0:
            tp = i,j
            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of bids for route j
AJ = []
k = 0
for l in route:
    A_temp = []
    for i in company:
        for j in route:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AJ.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AJ[0]))

# valid set of routes that can be covered by company i
AI = []
k = 0
for l in route:
    A_temp = []
    for i in company:
        for j in route:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp)
    k+=1

# take a look at a sample
# print(np.matrix(AI[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.BINARY, name='Assign')

# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)

# Cover each route
routeConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AJ[j]) >= 1 for j in route),
                                      name='routeConstrs')

# No company can be assigned more than two routes
companyConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[i]) <= 2 for i in company),
                                      name='companyConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 14 rows, 31 columns and 62 nonzeros
Model fingerprint: 0x9bb1731f
Variable types: 0 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 44300.000000
Presolve time: 0.00s
Presolved: 14 rows, 31 columns, 62 nonzeros
Variable types: 0 continuous, 31 integer (31 binary)

Root relaxation: objective 4.030000e+04, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [4]:
#####################################################
#         Assignment results
#####################################################

print(f"\n\n___Optimal assignment on each route________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        print("Company %2d is assigned to route %2d: $%4d" % (i+1, j+1, c[i][j]))
        t_cost += c[i][j]

print("The total cost of covering all routes is $%5d" % (t_cost))



___Optimal assignment on each route________
Company  1 is assigned to route  3: $7800
Company  2 is assigned to route  6: $3300
Company  2 is assigned to route  7: $4900
Company  3 is assigned to route  2: $4800
Company  5 is assigned to route  4: $3900
Company  5 is assigned to route  8: $3000
Company  6 is assigned to route  1: $7000
Company  6 is assigned to route  5: $5600
The total cost of covering all routes is $40300


--
##  Conclusion

In this example, we addressed the but route assignment problem. We determined the optimal assignment of companies to routes:
* Satisfy coverage for each route,
* Minimize the total operating cost, and
* Ensure no companies are assigned more than two routes, and no companies are assigned to a route where the is no bid.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid bids only. This benefit becomes more significant as problem size grows.

This bus assignment model can be used by many organizations to help make informed decisions about covering certain demands from a diverse set of resources where not all resources are suitable for all demands.
